**Instalando Pyspark**

In [ ]:
!pip install pyspark

**Montando o drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Importando Bibliotecas**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pandas as pd 

**Configurando a SparkSession**

In [ ]:
spark = (
    SparkSession.builder
      .master('local')
      .appName('dataframe_withcolumn')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

**Montando a estrutura do Dataframe**

In [ ]:
esquema = ["orgao_superior_codigo_siorg", "orgao_superior_nome", "orgao_nome", "ano_mes_referencia", "sigla_codigo_cargo_comissao", "cargos_distribuidos" ,"cargos_ocupados","cargos_vagos"]
df = (spark
       .read
       .format("csv")
       .option("header", "True")
       .option("inferschema", "True")
       .option("delimiter", ",")
       .load("/content/drive/MyDrive/projeto-aprimoramento-wander/df_tratado_pandas.csv").toDF(*esquema)
)

**Executando o spark para ver se esta funcionando corretamente**

In [ ]:
spark

**Exibindo o esquema do dataframe**

In [ ]:
df.printSchema()

root
 |-- orgao_superior_codigo_siorg: integer (nullable = true)
 |-- orgao_superior_nome: string (nullable = true)
 |-- orgao_nome: string (nullable = true)
 |-- ano_mes_referencia: integer (nullable = true)
 |-- sigla_codigo_cargo_comissao: string (nullable = true)
 |-- cargos_distribuidos: integer (nullable = true)
 |-- cargos_ocupados: integer (nullable = true)
 |-- cargos_vagos: integer (nullable = true)



In [ ]:
df.show()

+---------------------------+--------------------+--------------------+------------------+---------------------------+-------------------+---------------+------------+
|orgao_superior_codigo_siorg| orgao_superior_nome|          orgao_nome|ano_mes_referencia|sigla_codigo_cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------------------------+--------------------+--------------------+------------------+---------------------------+-------------------+---------------+------------+
|                         46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|       Direção e Assesso...|                 52|             52|           0|
|                         46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|       Direção e Assesso...|                 74|             74|           0|
|                         46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|       Direção e Assesso...|                 32|             28|      

**Renomeando algumas colunas do dataframe**

In [ ]:
df = df.withColumnRenamed('orgao_superior_codigo_siorg', 'cod_siorg').withColumnRenamed('orgao_superior_nome', 'orgao_superior').withColumnRenamed('orgao_nome','orgao_administrativo').withColumnRenamed('sigla_codigo_cargo_comissao','cargo_comissao')

In [ ]:
df.show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 52|             52|           0|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 74|             74|           0|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 32|             28|           4|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 17|             16|           1|

 **WHEN - OTHERWISE**

In [ ]:
#CRIANDO UMA NOVA COLUNA "STATUS" RELACIONANDO Á COLUNA CARGOS VAGOS
df1 = (df.withColumn("status", F.when(F.col("cargos_vagos") > 0, "Tem cargos vagos")
          .otherwise("Não tem cargos vagos"))    
      )

In [ ]:
df1.show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+--------------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|              status|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+--------------------+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 52|             52|           0|Não tem cargos vagos|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 74|             74|           0|Não tem cargos vagos|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 32|             28|           4|    Tem cargos vagos|
|       46|ADVOC

**GroupBy**

In [ ]:
#MIN - RETORNA O VALOR IGUAL OU MENOR A 0 DE CARGOS VAGOS RELACIONANDO A COLUNA CARGO DE COMISSÃO

df.groupBy(F.col("cargo_comissao")).min("cargos_vagos").show()

+--------------------+-----------------+
|      cargo_comissao|min(cargos_vagos)|
+--------------------+-----------------+
|Cargo de Natureza...|               -1|
|Funções Comission...|               -1|
|    Cargo de Direção|              -54|
|  Função Gratificada|               -1|
|Direção e Assesso...|               -8|
|Função Comissiona...|               -8|
|Vice-Presidente d...|                0|
|Cargo Comissionad...|                0|
|  Ministro de Estado|               -1|
|Cargo Comissionad...|               -1|
|Cargo Comissionad...|                0|
|Presidente da Rep...|                0|
|Comissionado de A...|               -1|
|Cargo Comissionad...|                0|
|Função Gratificad...|              -90|
|Funções Comission...|               -1|
+--------------------+-----------------+



In [ ]:
#MAX RETORAN O MAIOR NUMERO DE CARGOS VAGOS POR CARGO DE COMISSÃO
df.groupBy(F.col("cargo_comissao")).max("cargos_vagos").show()

+--------------------+-----------------+
|      cargo_comissao|max(cargos_vagos)|
+--------------------+-----------------+
|Cargo de Natureza...|                1|
|Funções Comission...|               63|
|    Cargo de Direção|                3|
|  Função Gratificada|              425|
|Direção e Assesso...|               48|
|Função Comissiona...|               74|
|Vice-Presidente d...|                0|
|Cargo Comissionad...|               12|
|  Ministro de Estado|                0|
|Cargo Comissionad...|                1|
|Cargo Comissionad...|                2|
|Presidente da Rep...|                0|
|Comissionado de A...|                3|
|Cargo Comissionad...|                3|
|Função Gratificad...|               68|
|Funções Comission...|              100|
+--------------------+-----------------+



In [ ]:
#COUNT RETORNA A QUANTIDADE DE CARGOS DE COMISSÃO 
df.groupBy(F.col("cargo_comissao")).count().show()

+--------------------+-----+
|      cargo_comissao|count|
+--------------------+-----+
|Cargo de Natureza...|   26|
|Funções Comission...|  174|
|    Cargo de Direção|  422|
|  Função Gratificada|  135|
|Direção e Assesso...|  562|
|Função Comissiona...|  102|
|Vice-Presidente d...|    1|
|Cargo Comissionad...|   51|
|  Ministro de Estado|   18|
|Cargo Comissionad...|   16|
|Cargo Comissionad...|   20|
|Presidente da Rep...|    1|
|Comissionado de A...|   23|
|Cargo Comissionad...|   41|
|Função Gratificad...|  344|
|Funções Comission...|  368|
+--------------------+-----+



In [ ]:
#SUM SOMA A QUANTIDADE DISTRIBUIDOS POR CARGOS DE COMISSÃO 
df.groupBy(F.col("cargo_comissao")).sum("cargos_distribuidos").show()

+--------------------+------------------------+
|      cargo_comissao|sum(cargos_distribuidos)|
+--------------------+------------------------+
|Cargo de Natureza...|                      47|
|Funções Comission...|                    5182|
|    Cargo de Direção|                    8383|
|  Função Gratificada|                   16813|
|Direção e Assesso...|                   10177|
|Função Comissiona...|                   12007|
|Vice-Presidente d...|                       1|
|Cargo Comissionad...|                    2562|
|  Ministro de Estado|                      18|
|Cargo Comissionad...|                     119|
|Cargo Comissionad...|                      47|
|Presidente da Rep...|                       1|
|Comissionado de A...|                     178|
|Cargo Comissionad...|                     643|
|Função Gratificad...|                   22464|
|Funções Comission...|                   13526|
+--------------------+------------------------+



**FILTER**

In [ ]:
#FILTRANDO A QUANTIDADE DE CARGOS POR DEPARTAMENTO (cargos de natureza especial)
Cargo_de_Natureza_Especial = df.filter(F.col('cargo_comissao')=='Cargo de Natureza Especial').show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Cargo de Natureza...|                  6|              6|           0|
|   214460|CONTROLADORIA-GER...|CONTROLADORIA-GER...|            202110|Cargo de Natureza...|                  1|              1|           0|
|       14|MINISTÉRIO DA AGR...|MINISTÉRIO DA AGR...|            202110|Cargo de Natureza...|                  2|              2|           0|
|   235874|MINISTÉRIO DA CID...|MINISTÉRIO DA CID...|            202110|Cargo de Natureza...|                  3|              3|           0|

In [ ]:
#FILTRANDO A QUANTIDADE DE CARGOS POR DEPARTAMENTO (cargos presidente da república)
Presidente_da_República = df.filter(F.col('cargo_comissao')=='Presidente da República').show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|       26|PRESIDÊNCIA DA RE...|PRESIDÊNCIA DA RE...|            202110|Presidente da Rep...|                  1|              1|           0|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+



In [ ]:
#FILTRANDO A QUANTIDADE DE CARGOS POR DEPARTAMENTO (cargos comissionado de assistência)
Cargo_Comissionado_de_Assistência = df.filter(F.col('cargo_comissao')=='Cargo Comissionado de Assistência').show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+
|     1988|MINISTÉRIO DA CIÊ...|AGÊNCIA NACIONAL ...|            202110|Cargo Comissionad...|                  2|              2|           0|
|     1988|MINISTÉRIO DA CIÊ...|AGÊNCIA NACIONAL ...|            202110|Cargo Comissionad...|                  1|              2|          -1|
|     2846|MINISTÉRIO DA INF...|AGÊNCIA NACIONAL ...|            202110|Cargo Comissionad...|                 19|             19|           0|
|     2846|MINISTÉRIO DA INF...|AGÊNCIA NACIONAL ...|            202110|Cargo Comissionad...|                 30|             29|           1|

**WHERE**

In [ ]:
#FILTRANDO A QUANTIDADE DE CARGOS VAGOS POR SETOR SE O VALOR DE CARGOS FOR IGUAL A 10
df.select(['cargo_comissao','cargos_vagos']).where(df.cargos_vagos == 10).show()

+--------------------+------------+
|      cargo_comissao|cargos_vagos|
+--------------------+------------+
|Funções Comission...|          10|
|  Função Gratificada|          10|
|  Função Gratificada|          10|
|Direção e Assesso...|          10|
|Função Gratificad...|          10|
|Função Comissiona...|          10|
|Cargo Comissionad...|          10|
|Funções Comission...|          10|
|  Função Gratificada|          10|
|Direção e Assesso...|          10|
+--------------------+------------+



In [ ]:
#FILTRANDO OS CARGOS DE COMISSÃO CUJO OS CARGOS VAGOS ESTÃO NEGATIVADOS
df2 = df.select(['cargo_comissao','cargos_vagos']).where(df.cargos_vagos <= -1)
df2.show()

+--------------------+------------+
|      cargo_comissao|cargos_vagos|
+--------------------+------------+
|Comissionado de A...|          -1|
|Cargo Comissionad...|          -1|
|Direção e Assesso...|          -3|
|  Ministro de Estado|          -1|
|Cargo de Natureza...|          -1|
|Função Comissiona...|          -1|
|    Cargo de Direção|          -1|
|    Cargo de Direção|          -8|
|    Cargo de Direção|          -5|
|    Cargo de Direção|         -29|
|Função Gratificad...|         -20|
|    Cargo de Direção|          -1|
|Função Gratificad...|          -1|
|    Cargo de Direção|          -1|
|    Cargo de Direção|          -1|
|Função Comissiona...|          -2|
|Função Gratificad...|          -1|
|Função Gratificad...|          -1|
|Função Gratificad...|          -1|
|Função Gratificad...|          -1|
+--------------------+------------+
only showing top 20 rows



**OrderBy**

In [ ]:
#CLASSIFICANDO A COLUNA "Cargos distribuidos" POR ORDEM DESCEDENTE
df.orderBy(F.col("cargos_distribuidos").desc()).show(truncate=False)

+---------+---------------------------------------------------+--------------------------------------------------------+------------------+------------------------------------------------------+-------------------+---------------+------------+
|cod_siorg|orgao_superior                                     |orgao_administrativo                                    |ano_mes_referencia|cargo_comissao                                        |cargos_distribuidos|cargos_ocupados|cargos_vagos|
+---------+---------------------------------------------------+--------------------------------------------------------+------------------+------------------------------------------------------+-------------------+---------------+------------+
|235876   |MINISTÉRIO DA ECONOMIA                             |MINISTÉRIO DA ECONOMIA                                  |202110            |Função Gratificada                                    |1615               |1388           |227         |
|276368   |MINISTÉRIO DO

**WindowFunctions**

In [ ]:
#row_number Window Function
#CRIANDO UMA COLUNA "LINHA" QUE RETORNA O NÚMERO DA LINHA DE ACORDO COM A COLUNA PARTICIONADA
w1 = Window.partitionBy(F.col("orgao_administrativo")).orderBy("cargos_distribuidos", "cargos_ocupados", "cargos_vagos")
df.withColumn("linha", F.row_number().over(w1)).show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+-----+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|linha|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+-----+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                  1|              1|           0|    1|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|  Ministro de Estado|                  1|              1|           0|    2|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Funções Comission...|                  2|              2|           0|    3|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Funções Comission...|             

In [ ]:
#Window Aggregate Functions
#AGREGANDO VÁRIOS VALORES RELACIONANDO A COLUNA CARGO DE COMISSÃO
(df.withColumn("QTD", F.row_number().over(w1))
  .withColumn("média", F.avg(F.col("cargos_distribuidos")).over(w1))
 .withColumn("soma", F.sum(F.col("cargos_distribuidos")).over(w1))
 .withColumn("min", F.min(F.col("cargos_distribuidos")).over(w1))
 .withColumn("max", F.max(F.col("cargos_distribuidos")).over(w1))
 .where(F.col("QTD") == 1).select("QTD","cargo_comissao", "média", "soma", "min", "max")
).show()

+---+--------------------+-----+----+---+---+
|QTD|      cargo_comissao|média|soma|min|max|
+---+--------------------+-----+----+---+---+
|  1|Direção e Assesso...|  1.0|   2|  1|  1|
|  1|Direção e Assesso...|  1.0|   3|  1|  1|
|  1|Cargo Comissionad...|  1.0|   2|  1|  1|
|  1|Cargo Comissionad...|  1.0|   2|  1|  1|
|  1|Cargo Comissionad...|  1.0|   2|  1|  1|
|  1|Comissionado de A...|  1.0|   3|  1|  1|
|  1|Cargo Comissionad...|  1.0|   2|  1|  1|
|  1|Cargo Comissionad...|  1.0|   1|  1|  1|
|  1|Cargo Comissionad...|  1.0|   1|  1|  1|
|  1|Cargo Comissionad...|  1.0|   1|  1|  1|
|  1|Cargo Comissionad...|  1.0|   2|  1|  1|
|  1|Cargo Comissionad...|  1.0|   1|  1|  1|
|  1|Cargo Comissionad...|  1.0|   1|  1|  1|
|  1|Direção e Assesso...|  1.0|   4|  1|  1|
|  1|    Cargo de Direção|  1.0|   1|  1|  1|
|  1|    Cargo de Direção|  1.0|   1|  1|  1|
|  1|    Cargo de Direção|  1.0|   1|  1|  1|
|  1|    Cargo de Direção|  1.0|  12|  1|  1|
|  1|Cargo de Natureza...|  1.0|  

In [ ]:
#percent_rank Window Function
#CRIANDO UM COLUNA QUE RETORNA A PORCENTAGEM DE CARGOS DISTRIBUIDOS NA ORDEM ASCEDENTE
df.withColumn("percent_rank", F.percent_rank().over(w1)).show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+-------------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|       percent_rank|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+-------------------+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                  1|              1|           0|                0.0|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|  Ministro de Estado|                  1|              1|           0|                0.0|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Funções Comission...|                  2|              2|           0|0.08695652173913043|
|       46|ADVOCACIA-G

**Criando duas novas coluna referindo a porcentagem em relação a coluna cargos ocupados e cargos vagos**

In [ ]:
df = df.withColumn("porcentagem_cargos_ocupados", F.col("cargos_ocupados") * 100 / F.col("cargos_distribuidos"))
df = df.withColumn("porcentagem_cargos_vagos", F.col("cargos_vagos") * 100 / F.col("cargos_distribuidos"))

In [ ]:
df.show()

+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+---------------------------+------------------------+
|cod_siorg|      orgao_superior|orgao_administrativo|ano_mes_referencia|      cargo_comissao|cargos_distribuidos|cargos_ocupados|cargos_vagos|porcentagem_cargos_ocupados|porcentagem_cargos_vagos|
+---------+--------------------+--------------------+------------------+--------------------+-------------------+---------------+------------+---------------------------+------------------------+
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 52|             52|           0|                      100.0|                     0.0|
|       46|ADVOCACIA-GERAL D...|ADVOCACIA-GERAL D...|            202110|Direção e Assesso...|                 74|             74|           0|                      100.0|                     0.0|
|       46|ADVOCACIA

Convertendo o arquivo em pandas e salvando em csv

In [ ]:
df_conv = df.toPandas()

In [ ]:
df_conv.to_csv('df_tratado_pyspark.csv',index=False)